In [92]:
import datetime as dt

import requests
import pandas as pd
import numpy as np
import matplotlib
import json    
import requests


In [93]:
resp = requests.get('https://api.kraken.com/0/public/AssetPairs?')
data = resp.json()
def getmarkets(data):
    pairs = []
    navne = list(data["result"].keys())
    for navn in navne:
        if navn[-3:] == "USD":
            pairs.append((navn))
    return pairs


perpnames = getmarkets(data)

In [94]:
import datetime as dt
# get today's datetime
input_dt = dt.datetime.today()
print('Datetime is:', input_dt)

res = input_dt.replace(day=1, hour=0, minute = 0, second = 0, microsecond = 0)
print('Only date:', res.timestamp())


Datetime is: 2024-04-10 16:23:49.556576
Only date: 1711922400.0


In [95]:
def now(period:str): 
    input_dt = dt.datetime.today()
    if period == "monthly":
        res = input_dt.replace(day=1, hour=0, minute = 0, second = 0, microsecond = 0)
    if period == "yearly":
        res = input_dt.replace(month = 1, day=1, hour=0, minute = 0, second = 0, microsecond = 0)
    if period == "quarterly":
        month = (input_dt.month-1)//3*3+1
        res = input_dt.replace(month = month, day=1, hour=0, minute = 0, second = 0, microsecond = 0)
        
    unix = res.timestamp()
    return int(unix)
    

In [96]:
def ohlc(name, tf, period):
    lookback = now(period) 
    data = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={name}&interval={tf}&since={lookback}') 
    data = data.json()
    data = pd.DataFrame(data['result'])
    # Drop the last column
    data = data.iloc[:, :-1]
    
    
    return data

In [97]:
def modify_list(lst):
    ny_list = []
    high = lst[2]
    low = lst[3]
    close = lst[4]
    vwap = lst[5]
    volume = lst[6]
    high = float(high)
    low = float(low)
    close = float(close)
    volume = float(volume)
    
    ny_list.append(high)
    ny_list.append(low)
    ny_list.append(close)
    ny_list.append(volume)
    return ny_list

In [98]:
dfs = []
tf = 1440
period = "yearly"
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, period))

In [99]:
df = pd.concat(dfs, axis=1)
df = df.dropna(axis="columns")
modified_df = df.applymap(modify_list)


modified_df.reset_index(drop=True, inplace=True)
#search_result = df.applymap(lambda x: not isinstance(x, list)).any().any()
#search_result1 = df.isin(["NaN"]).any().any()

df = pd.concat(dfs, axis=1)
df = df.dropna(axis="columns")
modified_df = df.applymap(modify_list)

removed_columns = []
for column in modified_df.columns:
    if any(lst[3] == 0 for lst in modified_df[column]):
        removed_columns.append(column)
        modified_df.drop(column, axis=1, inplace=True)

print("Removed columns:", removed_columns)

modified_df.reset_index(drop=True, inplace=True)
#search_result = df.applymap(lambda x: not isinstance(x, list)).any().any()
#search_result1 = df.isin(["NaN"]).any().any()


Removed columns: ['ETHPYUSD', 'POLUSD', 'REQUSD', 'TBTCUSD']


In [100]:
def get_vwap(data):
    transformed = data.to_numpy()
    x, y = np.shape(transformed)
    for j in range(y):
        total_value = 0
        total_volume = 0
        for i in range(x):

            # Calculate typical price
            typ_price = ((transformed[i][j][0] + transformed[i][j][1] + transformed[i][j][2]) / 3) * transformed[i][j][3] 
            # Calculate the total traded value for this row
            total_value += typ_price # Assuming the volume is at index 3
            # Calculate the total volume for this column
            total_volume += transformed[i][j][3]  # Assuming the volume is at index 3
            vwap = total_value / total_volume
            transformed[i][j].append(vwap)
        
    df = pd.DataFrame(transformed, index=data.index, columns=data.columns)
    return df



In [101]:
x = get_vwap(modified_df)
x


,1INCHUSD,AAVEUSD,ACAUSD,ACHUSD,ADAUSD,ADXUSD,AGLDUSD,AIRUSD,AKTUSD,ALCXUSD,...,XXBTZUSD,XXLMZUSD,XXMRZUSD,XXRPZUSD,XZECZUSD,YFIUSD,YGGUSD,ZEURZUSD,ZGBPZUSD,ZRXUSD
0,"[0.544, 0.428, 0.488, 1387709.51799248, 0.4866...","[116.27, 108.0, 116.2, 5758.02218684, 113.4899...","[0.106, 0.096, 0.106, 195997.63243447, 0.10266...","[0.02242, 0.0207, 0.02234, 1988219.76939657, 0...","[0.62353, 0.590758, 0.622753, 5442214.98829888...","[0.19768, 0.19229, 0.1964, 2711.29652595, 0.19...","[1.5516, 1.2212, 1.5345, 8259.5948952, 1.43576...","[0.02067, 0.0188, 0.01961, 1641879.95609134, 0...","[2.53, 2.3705, 2.3983, 272976.23980653, 2.4329...","[42.26, 31.37, 32.17, 5882.72570559, 35.266666...",...,"[44187.5, 42215.7, 44180.8, 2457.30419634, 435...","[0.132413, 0.127313, 0.131879, 3414786.2974095...","[169.53, 163.2, 169.43, 4520.39184311, 167.386...","[0.63118, 0.60786, 0.62968, 7073603.963705, 0....","[27.89, 26.47, 27.82, 5731.15089471, 27.393333...","[8279.0, 7936.0, 8273.0, 8.16789928, 8162.6666...","[0.45, 0.414, 0.448, 25054.52463167, 0.4373333...","[1.10395, 1.09984, 1.10255, 6162485.4793237, 1...","[1.27311, 1.26801, 1.27061, 1032444.39956221, ...","[0.376, 0.362, 0.374, 125851.90007875, 0.37066..."
1,"[0.496, 0.466, 0.474, 742031.53298379, 0.48387...","[119.92, 108.97, 110.31, 10530.30587669, 113.2...","[0.129, 0.104, 0.114, 435412.0844429, 0.111631...","[0.02272, 0.02163, 0.02178, 2445900.21125193, ...","[0.637184, 0.602503, 0.60486, 8599439.7865338,...","[0.19873, 0.19428, 0.19428, 3478.60023988, 0.1...","[1.6668, 1.4366, 1.4569, 7643.84447587, 1.4763...","[0.0235, 0.0188, 0.02093, 2337422.37947706, 0....","[2.5103, 2.2844, 2.3249, 300259.31627019, 2.40...","[36.0, 31.76, 34.15, 3281.5180705, 34.80235829...",...,"[45913.4, 44113.2, 44972.0, 6462.5537177, 4459...","[0.133345, 0.128589, 0.129124, 3602227.3900236...","[174.36, 167.02, 167.57, 10627.77872584, 168.9...","[0.64, 0.62187, 0.62461, 10843095.01748708, 0....","[28.31, 27.17, 27.56, 6332.56932869, 27.543812...","[8423.0, 8117.0, 8227.0, 24.9796386, 8232.7504...","[0.489, 0.44, 0.481, 135887.23808026, 0.464914...","[1.10414, 1.08601, 1.09359, 25114563.27966746,...","[1.27499, 1.26043, 1.26181, 5248704.48025492, ...","[0.383, 0.365, 0.372, 91222.46803823, 0.371787..."
2,"[0.486, 0.335, 0.425, 621056.64574245, 0.46840...","[113.04, 91.0, 102.72, 14343.78803955, 108.082...","[0.124, 0.1, 0.107, 618445.54031363, 0.1109890...","[0.02293, 0.01821, 0.01974, 8042875.67507256, ...","[0.618626, 0.520006, 0.557302, 19259636.722844...","[0.1965, 0.17954, 0.18566, 20149.75688852, 0.1...","[1.469, 1.0865, 1.1718, 25616.03097509, 1.3320...","[0.02158, 0.01889, 0.01965, 1636751.86611507, ...","[2.661, 2.1747, 2.5473, 559003.26151445, 2.430...","[34.44, 25.0, 28.76, 3104.23507626, 33.4354251...",...,"[45509.3, 39501.0, 42849.5, 7003.5579979, 4372...","[0.132894, 0.111584, 0.12058, 10050065.8466747...","[169.59, 150.4, 153.94, 18164.49468008, 162.97...","[0.6394, 0.45411, 0.58252, 39632425.1845478, 0...","[28.25, 22.6, 25.14, 15139.54773227, 26.311750...","[8300.0, 6960.0, 7487.0, 89.34243528, 7758.345...","[0.569, 0.374, 0.53, 478031.15001857, 0.484429...","[1.0964, 1.086, 1.09094, 35347259.69326952, 1....","[1.26593, 1.25897, 1.26574, 7902062.23008546, ...","[0.377, 0.285, 0.33, 535296.49528109, 0.342530..."
3,"[0.483, 0.415, 0.458, 319518.92009287, 0.46669...","[105.5, 101.41, 104.04, 4646.58543163, 107.498...","[0.122, 0.107, 0.111, 442180.34536418, 0.11160...","[0.02032, 0.01918, 0.01983, 2192139.55393805, ...","[0.579461, 0.549955, 0.570307, 7250994.1872421...","[0.19532, 0.18547, 0.19488, 10362.86642748, 0....","[1.2634, 1.1458, 1.2537, 5042.39732507, 1.3199...","[0.02146, 0.01902, 0.02058, 1150937.19790741, ...","[2.9289, 2.5061, 2.9113, 408932.09783151, 2.52...","[34.52, 27.45, 31.7, 1881.77575662, 33.1412494...",...,"[44784.9, 42682.0, 44211.9, 3461.20649181, 437...","[0.12217, 0.119413, 0.121657, 5070136.22342526...","[165.01, 150.0, 164.87, 12764.91469194, 162.14...","[0.59428, 0.570

In [102]:
def get_bands(data):
    transformed = data.to_numpy()

    x,y = np.shape(transformed)
    for j in range(y):
        looked = []
        for i in range(x):
            looked.append(transformed[i][j][1])
            std_dev = np.std(looked)
            transformed[i][j].append(transformed[i][j][4] + std_dev)
            transformed[i][j].append(transformed[i][j][4] - std_dev)
    df = pd.DataFrame(transformed, index=data.index, columns=data.columns)
    return df
        

In [103]:
x = get_bands(modified_df)
x

,1INCHUSD,AAVEUSD,ACAUSD,ACHUSD,ADAUSD,ADXUSD,AGLDUSD,AIRUSD,AKTUSD,ALCXUSD,...,XXBTZUSD,XXLMZUSD,XXMRZUSD,XXRPZUSD,XZECZUSD,YFIUSD,YGGUSD,ZEURZUSD,ZGBPZUSD,ZRXUSD
0,"[0.544, 0.428, 0.488, 1387709.51799248, 0.4866...","[116.27, 108.0, 116.2, 5758.02218684, 113.4899...","[0.106, 0.096, 0.106, 195997.63243447, 0.10266...","[0.02242, 0.0207, 0.02234, 1988219.76939657, 0...","[0.62353, 0.590758, 0.622753, 5442214.98829888...","[0.19768, 0.19229, 0.1964, 2711.29652595, 0.19...","[1.5516, 1.2212, 1.5345, 8259.5948952, 1.43576...","[0.02067, 0.0188, 0.01961, 1641879.95609134, 0...","[2.53, 2.3705, 2.3983, 272976.23980653, 2.4329...","[42.26, 31.37, 32.17, 5882.72570559, 35.266666...",...,"[44187.5, 42215.7, 44180.8, 2457.30419634, 435...","[0.132413, 0.127313, 0.131879, 3414786.2974095...","[169.53, 163.2, 169.43, 4520.39184311, 167.386...","[0.63118, 0.60786, 0.62968, 7073603.963705, 0....","[27.89, 26.47, 27.82, 5731.15089471, 27.393333...","[8279.0, 7936.0, 8273.0, 8.16789928, 8162.6666...","[0.45, 0.414, 0.448, 25054.52463167, 0.4373333...","[1.10395, 1.09984, 1.10255, 6162485.4793237, 1...","[1.27311, 1.26801, 1.27061, 1032444.39956221, ...","[0.376, 0.362, 0.374, 125851.90007875, 0.37066..."
1,"[0.496, 0.466, 0.474, 742031.53298379, 0.48387...","[119.92, 108.97, 110.31, 10530.30587669, 113.2...","[0.129, 0.104, 0.114, 435412.0844429, 0.111631...","[0.02272, 0.02163, 0.02178, 2445900.21125193, ...","[0.637184, 0.602503, 0.60486, 8599439.7865338,...","[0.19873, 0.19428, 0.19428, 3478.60023988, 0.1...","[1.6668, 1.4366, 1.4569, 7643.84447587, 1.4763...","[0.0235, 0.0188, 0.02093, 2337422.37947706, 0....","[2.5103, 2.2844, 2.3249, 300259.31627019, 2.40...","[36.0, 31.76, 34.15, 3281.5180705, 34.80235829...",...,"[45913.4, 44113.2, 44972.0, 6462.5537177, 4459...","[0.133345, 0.128589, 0.129124, 3602227.3900236...","[174.36, 167.02, 167.57, 10627.77872584, 168.9...","[0.64, 0.62187, 0.62461, 10843095.01748708, 0....","[28.31, 27.17, 27.56, 6332.56932869, 27.543812...","[8423.0, 8117.0, 8227.0, 24.9796386, 8232.7504...","[0.489, 0.44, 0.481, 135887.23808026, 0.464914...","[1.10414, 1.08601, 1.09359, 25114563.27966746,...","[1.27499, 1.26043, 1.26181, 5248704.48025492, ...","[0.383, 0.365, 0.372, 91222.46803823, 0.371787..."
2,"[0.486, 0.335, 0.425, 621056.64574245, 0.46840...","[113.04, 91.0, 102.72, 14343.78803955, 108.082...","[0.124, 0.1, 0.107, 618445.54031363, 0.1109890...","[0.02293, 0.01821, 0.01974, 8042875.67507256, ...","[0.618626, 0.520006, 0.557302, 19259636.722844...","[0.1965, 0.17954, 0.18566, 20149.75688852, 0.1...","[1.469, 1.0865, 1.1718, 25616.03097509, 1.3320...","[0.02158, 0.01889, 0.01965, 1636751.86611507, ...","[2.661, 2.1747, 2.5473, 559003.26151445, 2.430...","[34.44, 25.0, 28.76, 3104.23507626, 33.4354251...",...,"[45509.3, 39501.0, 42849.5, 7003.5579979, 4372...","[0.132894, 0.111584, 0.12058, 10050065.8466747...","[169.59, 150.4, 153.94, 18164.49468008, 162.97...","[0.6394, 0.45411, 0.58252, 39632425.1845478, 0...","[28.25, 22.6, 25.14, 15139.54773227, 26.311750...","[8300.0, 6960.0, 7487.0, 89.34243528, 7758.345...","[0.569, 0.374, 0.53, 478031.15001857, 0.484429...","[1.0964, 1.086, 1.09094, 35347259.69326952, 1....","[1.26593, 1.25897, 1.26574, 7902062.23008546, ...","[0.377, 0.285, 0.33, 535296.49528109, 0.342530..."
3,"[0.483, 0.415, 0.458, 319518.92009287, 0.46669...","[105.5, 101.41, 104.04, 4646.58543163, 107.498...","[0.122, 0.107, 0.111, 442180.34536418, 0.11160...","[0.02032, 0.01918, 0.01983, 2192139.55393805, ...","[0.579461, 0.549955, 0.570307, 7250994.1872421...","[0.19532, 0.18547, 0.19488, 10362.86642748, 0....","[1.2634, 1.1458, 1.2537, 5042.39732507, 1.3199...","[0.02146, 0.01902, 0.02058, 1150937.19790741, ...","[2.9289, 2.5061, 2.9113, 408932.09783151, 2.52...","[34.52, 27.45, 31.7, 1881.77575662, 33.1412494...",...,"[44784.9, 42682.0, 44211.9, 3461.20649181, 437...","[0.12217, 0.119413, 0.121657, 5070136.22342526...","[165.01, 150.0, 164.87, 12764.91469194, 162.14...","[0.59428, 0.570

In [104]:
def scanner(data):
    transformed = data.to_numpy()
    x,y = np.shape(transformed)
    for j in range(y):
        for i in range(x):
            if transformed[i][j][2] > transformed[i][j][5]:
                transformed[i][j].append(1)
            elif transformed[i][j][2] < transformed[i][j][6]:
                transformed[i][j].append(-1)
    df = pd.DataFrame(transformed, index=data.index, columns=data.columns)
    return df


In [105]:
z = scanner(x)
z

,1INCHUSD,AAVEUSD,ACAUSD,ACHUSD,ADAUSD,ADXUSD,AGLDUSD,AIRUSD,AKTUSD,ALCXUSD,...,XXBTZUSD,XXLMZUSD,XXMRZUSD,XXRPZUSD,XZECZUSD,YFIUSD,YGGUSD,ZEURZUSD,ZGBPZUSD,ZRXUSD
0,"[0.544, 0.428, 0.488, 1387709.51799248, 0.4866...","[116.27, 108.0, 116.2, 5758.02218684, 113.4899...","[0.106, 0.096, 0.106, 195997.63243447, 0.10266...","[0.02242, 0.0207, 0.02234, 1988219.76939657, 0...","[0.62353, 0.590758, 0.622753, 5442214.98829888...","[0.19768, 0.19229, 0.1964, 2711.29652595, 0.19...","[1.5516, 1.2212, 1.5345, 8259.5948952, 1.43576...","[0.02067, 0.0188, 0.01961, 1641879.95609134, 0...","[2.53, 2.3705, 2.3983, 272976.23980653, 2.4329...","[42.26, 31.37, 32.17, 5882.72570559, 35.266666...",...,"[44187.5, 42215.7, 44180.8, 2457.30419634, 435...","[0.132413, 0.127313, 0.131879, 3414786.2974095...","[169.53, 163.2, 169.43, 4520.39184311, 167.386...","[0.63118, 0.60786, 0.62968, 7073603.963705, 0....","[27.89, 26.47, 27.82, 5731.15089471, 27.393333...","[8279.0, 7936.0, 8273.0, 8.16789928, 8162.6666...","[0.45, 0.414, 0.448, 25054.52463167, 0.4373333...","[1.10395, 1.09984, 1.10255, 6162485.4793237, 1...","[1.27311, 1.26801, 1.27061, 1032444.39956221, ...","[0.376, 0.362, 0.374, 125851.90007875, 0.37066..."
1,"[0.496, 0.466, 0.474, 742031.53298379, 0.48387...","[119.92, 108.97, 110.31, 10530.30587669, 113.2...","[0.129, 0.104, 0.114, 435412.0844429, 0.111631...","[0.02272, 0.02163, 0.02178, 2445900.21125193, ...","[0.637184, 0.602503, 0.60486, 8599439.7865338,...","[0.19873, 0.19428, 0.19428, 3478.60023988, 0.1...","[1.6668, 1.4366, 1.4569, 7643.84447587, 1.4763...","[0.0235, 0.0188, 0.02093, 2337422.37947706, 0....","[2.5103, 2.2844, 2.3249, 300259.31627019, 2.40...","[36.0, 31.76, 34.15, 3281.5180705, 34.80235829...",...,"[45913.4, 44113.2, 44972.0, 6462.5537177, 4459...","[0.133345, 0.128589, 0.129124, 3602227.3900236...","[174.36, 167.02, 167.57, 10627.77872584, 168.9...","[0.64, 0.62187, 0.62461, 10843095.01748708, 0....","[28.31, 27.17, 27.56, 6332.56932869, 27.543812...","[8423.0, 8117.0, 8227.0, 24.9796386, 8232.7504...","[0.489, 0.44, 0.481, 135887.23808026, 0.464914...","[1.10414, 1.08601, 1.09359, 25114563.27966746,...","[1.27499, 1.26043, 1.26181, 5248704.48025492, ...","[0.383, 0.365, 0.372, 91222.46803823, 0.371787..."
2,"[0.486, 0.335, 0.425, 621056.64574245, 0.46840...","[113.04, 91.0, 102.72, 14343.78803955, 108.082...","[0.124, 0.1, 0.107, 618445.54031363, 0.1109890...","[0.02293, 0.01821, 0.01974, 8042875.67507256, ...","[0.618626, 0.520006, 0.557302, 19259636.722844...","[0.1965, 0.17954, 0.18566, 20149.75688852, 0.1...","[1.469, 1.0865, 1.1718, 25616.03097509, 1.3320...","[0.02158, 0.01889, 0.01965, 1636751.86611507, ...","[2.661, 2.1747, 2.5473, 559003.26151445, 2.430...","[34.44, 25.0, 28.76, 3104.23507626, 33.4354251...",...,"[45509.3, 39501.0, 42849.5, 7003.5579979, 4372...","[0.132894, 0.111584, 0.12058, 10050065.8466747...","[169.59, 150.4, 153.94, 18164.49468008, 162.97...","[0.6394, 0.45411, 0.58252, 39632425.1845478, 0...","[28.25, 22.6, 25.14, 15139.54773227, 26.311750...","[8300.0, 6960.0, 7487.0, 89.34243528, 7758.345...","[0.569, 0.374, 0.53, 478031.15001857, 0.484429...","[1.0964, 1.086, 1.09094, 35347259.69326952, 1....","[1.26593, 1.25897, 1.26574, 7902062.23008546, ...","[0.377, 0.285, 0.33, 535296.49528109, 0.342530..."
3,"[0.483, 0.415, 0.458, 319518.92009287, 0.46669...","[105.5, 101.41, 104.04, 4646.58543163, 107.498...","[0.122, 0.107, 0.111, 442180.34536418, 0.11160...","[0.02032, 0.01918, 0.01983, 2192139.55393805, ...","[0.579461, 0.549955, 0.570307, 7250994.1872421...","[0.19532, 0.18547, 0.19488, 10362.86642748, 0....","[1.2634, 1.1458, 1.2537, 5042.39732507, 1.3199...","[0.02146, 0.01902, 0.02058, 1150937.19790741, ...","[2.9289, 2.5061, 2.9113, 408932.09783151, 2.52...","[34.52, 27.45, 31.7, 1881.77575662, 33.1412494...",...,"[44784.9, 42682.0, 44211.9, 3461.20649181, 437...","[0.12217, 0.119413, 0.121657, 5070136.22342526...","[165.01, 150.0, 164.87, 12764.91469194, 162.14...","[0.59428, 0.570

In [106]:
bands_df = z.tail(1)
bullish = []
for col in z.columns:
    if any(1 in sublist for sublist in bands_df[col]):
        bullish.append(col)
bearish = []
for col in z.columns:
    if any(-1 in sublist for sublist in bands_df[col]):
        bearish.append(col)


In [107]:
columns = (["Bullish", "Bearish"])
# Determine the length of the longest list
max_length = max(len(bullish), len(bearish))

# Create a dictionary to hold the data
data = {'bullish': bullish + [None] * (max_length - len(bullish)),
        'bearish': bearish + [None] * (max_length - len(bearish))}
website = pd.DataFrame(data)
website

,bullish,bearish
0,ALCXUSD,ARBUSD
1,ANKRUSD,ASTRUSD
2,ARPAUSD,AUDIOUSD
3,BCHUSD,BLURUSD
4,BICOUSD,BNCUSD
5,BITUSD,BSXUSD
6,BLZUSD,EGLDUSD
7,BRICKUSD,EURTUSD
8,CELRUSD,FXSUSD
9,CHZUSD,GARIUSD


In [108]:
def detect_vwap_slope(data):
    transformed = data.to_numpy()
    x, y = np.shape(transformed)
    slopes = []
    threshold = 0.95
    test = []
    
    for j in range(y):
        vwap_values = []
        
        for i in range(x):
            vwap_values.append(transformed[i][j][4])  # Assuming VWAP is stored in the fourth column
            
        slope = np.gradient(vwap_values)
        
        # Min-Max normalization of the slope
        min_slope = np.min(slope)
        max_slope = np.max(slope)
        norm_slope = (slope - min_slope) / (max_slope - min_slope)
        
        slopes.append(norm_slope)

    # Now you can compare the normalized slopes against your threshold
    # for slope in slopes:
    #     if (np.abs(slope)).all() < threshold:
    #         test.append("Flat")
    #     else:
    #         test.append("Sloped")
    return slopes


In [109]:
test = detect_vwap_slope(x)
test


[array([0.51369153, 0.        , 0.04373346, 0.75322705, 0.72573594,
        0.57048374, 0.49208856, 0.50587787, 0.53810202, 0.72131431,
        0.87831083, 0.77615325, 0.76598021, 0.74652417, 0.74964227,
        0.74716633, 0.72260697, 0.71081292, 0.72420663, 0.72906546,
        0.71080961, 0.6705822 , 0.68378789, 0.72173732, 0.71937848,
        0.71853197, 0.71621823, 0.7112505 , 0.71611492, 0.70607028,
        0.70274861, 0.66866189, 0.67130115, 0.7272021 , 0.70589913,
        0.69762958, 0.72272211, 0.72746716, 0.72007334, 0.72454969,
        0.73340111, 0.73118507, 0.73306671, 0.73472014, 0.73950591,
        0.74213828, 0.73753438, 0.73698346, 0.74038227, 0.73684603,
        0.7242153 , 0.72159081, 0.75545154, 0.85052759, 0.85518817,
        0.81522647, 0.81449817, 0.8067027 , 0.83109532, 0.91291288,
        0.91982426, 0.86715498, 0.90776167, 0.95965297, 1.        ,
        0.99819202, 0.90799012, 0.87724206, 0.86114214, 0.88841403,
        0.92284443, 0.98152803, 0.98886763, 0.84

In [110]:
zipped = zip(perpnames, test)
make_dict= dict(zipped)
make_df = pd.DataFrame(make_dict)
make_tail = make_df.tail(1)
make_tail["POLSUSD"]

100    0.628048
Name: POLSUSD, dtype: float64

In [111]:
def calculate_vwap_slope(data, window=20, threshold=1):
    vwap_slope_data = pd.DataFrame(index=data.index, columns=data.columns)
    for column in data.columns:
        vwap_values = data[column].apply(lambda x: x[4])  # Extract VWAP values from the fourth element of each list
        
        # Calculate the VWAP slope using the diff function
        vwap_slope = vwap_values.diff(periods=5)
        
        # Normalize the VWAP slope by dividing it by the long-term standard deviation
        vwap_slope_normalized = vwap_slope / vwap_slope.rolling(window=window).std()
        
        # Assign normalized VWAP slope values to the DataFrame only if they are greater than the threshold
        vwap_slope_data[column] = vwap_slope_normalized.where(abs(vwap_slope_normalized) > threshold, np.nan)
    
    
    return vwap_slope_data



In [112]:
slope = calculate_vwap_slope(x)
last_row = slope.tail(1)
last_row.dropna(axis=1, inplace=True)
# combined_df = pd.concat([website, last_row])
# combined_df


/var/folders/xw/j6c1c3514_l7lyj6wyc6l1rc0000gn/T/ipykernel_59829/896865054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_row.dropna(axis=1, inplace=True)


In [113]:
bullish_slope = []
for column in last_row.columns:
    if last_row[column].values[0] > 0:
        bullish_slope.append(column)
bearish_slope = []
for column in last_row.columns:
    if last_row[column].values[0] < 0:
        bearish_slope.append(column)

columns = (["Bullish slope", "Bearish slope"])
# Determine the length of the longest list
max_length = max(len(bullish_slope), len(bearish_slope))

# Create a dictionary to hold the data
sloped_data = {'bullish slope': bullish_slope + [None] * (max_length - len(bullish_slope)),
        'bearish slope': bearish_slope + [None] * (max_length - len(bearish_slope))}
sloped_df = pd.DataFrame(sloped_data)
sloped_df
combined_df = pd.concat([website, sloped_df])
# Merge for bullish
bullish_df = pd.merge(website, sloped_df, how='inner', left_on='bullish', right_on='bullish slope')

# Merge for bearish
bearish_df = pd.merge(website, sloped_df, how='inner', left_on='bearish', right_on='bearish slope')

# Concatenate the results
combined_filtered_df = pd.concat([bullish_df["bullish"], bearish_df["bearish"]], axis=1)
combined_filtered_df
df = combined_filtered_df[combined_filtered_df['bullish'].notna() & combined_filtered_df['bearish'].notna()]
df
# filtered_df = combined_df[combined_df['bullish'].isin(combined_df['bullish slope']) & combined_df['bearish'].isin(combined_df['bearish slope'])]
# filtered_df2 = combined_df[combined_df['bullish slope'].isin(combined_df['bullish']) & combined_df['bearish slope'].isin(combined_df['bearish'])]
# combined_filtered_df = pd.concat([filtered_df, filtered_df2])
# combined_filtered_df

,bullish,bearish
0,ALCXUSD,ARBUSD
1,BCHUSD,ASTRUSD
2,BICOUSD,BLURUSD
3,BLZUSD,BNCUSD
4,CELRUSD,GMXUSD
5,CHZUSD,INJUSD
6,ETHWUSD,JUNOUSD
7,FARMUSD,KARUSD
8,GALAUSD,KSMUSD
9,MNGOUSD,LDOUSD
